# Landuse and landcover dataset

Possible dataset

1. African land cover (20m)
https://www.copernicus.eu/en/media/images/african-land-cover
  Sentinel-2A images (2016)


2. Copernicus Global Land Cover Layers: CGLS-LC100 collection 3 (100m)
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global
  PROBA-V 100 m time-series 
  (2015-01-01 ~ 2019-12-31)


3. PML_V2: Coupled Evapotranspiration and Gross Primary Product (500m)
https://developers.google.com/earth-engine/datasets/catalog/CAS_IGSNRR_PML_V2
  Evapotranspiration(ET), Gross primary product(GPP)
  (2002-07-04 ~ 2017-12-27）


4. SRTM Digital Elevation Data Version 4 (90m)
https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4#terms-of-use
  elevation 
  (2000-02-11 ~ 2000-02-22)


5. GPWv411: Basic Demographic Characteristics (Gridded PopulatCSP gHM: Global Human Modificationion of the World Version 4.11) (1km)
https://developers.google.com/earth-engine/datasets/catalog/CIESIN_GPWv411_GPW_Basic_Demographic_Characteristics#description
  population
  (2000-01-01 ~ 2020-01-01)


6. CSP gHM: Global Human Modification (1km)
https://developers.google.com/earth-engine/datasets/catalog/CSP_HM_GlobalHumanModification#bands
  global human modification
  (2016-01-01 ~ 2016-12-31)


7. FireCCI51: MODIS Fire_cci Burned Area Pixel product, version 5.1 (0.00224573 arc degrees)
https://developers.google.com/earth-engine/datasets/catalog/ESA_CCI_FireCCI_5_1#description
  Burndate; Landcover of burned pixels
  (2011-01-01 ~ 2019-12-31)


8. GlobCover: Global Land Cover Map (300m)
https://developers.google.com/earth-engine/datasets/catalog/ESA_GLOBCOVER_L4_200901_200912_V2_3#bands
  landcover
  (2009-01-01 ~ 2010-01-01)


9. Global PALSAR-2/PALSAR Forest/Non-Forest Map (25m)
https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR_YEARLY_FNF
  forest and non-forest landcover classification
  (2007-01-01 ~ 2018-01-01)


10. MCD12Q1.006 MODIS Land Cover Type Yearly Global （500m） 
https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1#bands
  landcover
  (2001-01-01 ~ 2019-01-01)


11. Oxford MAP: Malaria Atlas Project Fractional International Geosphere-Biosphere Programme Landcover (5000m)
https://developers.google.com/earth-engine/datasets/catalog/Oxford_MAP_IGBP_Fractional_Landcover_5km_Annual
  landcover
  (2001-01-01 ~ 2013-01-01)


12. GFSAD1000: Cropland Extent 1km Multi-Study Crop Mask, Global Food-Support Analysis Data (1000m)
https://developers.google.com/earth-engine/datasets/catalog/USGS_GFSAD1000_V1
  landcover
  (2010)

## Load raster
Setting environment:
```
conda create -n gee python
conda activate gee
conda install mamba -c conda-forge
mamba install geemap xarray_leaflet -c conda-forge
```

* But this function doesn't work on windows system now

In [ ]:
import os
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

## Download an ee.Image

In [ ]:
import ee
import geemap
import os

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
image = ee.Image('LE7_TOA_5YEAR/1999_2003')

landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.4
}
Map.addLayer(image, landsat_vis, "LE7_TOA_5YEAR/1999_2003", True, 0.7)

In [ ]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature

if feature is None:
    geom = ee.Geometry.Polygon([[
    [-2.399214, 11.889637],
    [-2.399214, 12.626826],
    [-1.022788, 12.626826],
    [-1.022788, 11.889637],
    [-2.399214, 11.889637]]])
    feature = ee.Feature(geom, {})

roi = feature.geometry()
Map.addLayer(roi, {}, "ROI")

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'landsat.tif')
print(out_dir)

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
image = image.clip(roi).unmask()
geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=False)

In [ ]:
image = image.clip(roi).unmask()
geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=False)

In [ ]:
geemap.ee_export_image_to_drive(image, description='landsat', folder='export', region=roi, scale=30)

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

# Dano = ee.Geometry.Point(-3.06244, 11.146104)
collection = ee.ImageCollection("LANDSAT/LT05/C01/T2") \
    .filterBounds(bf_boarder) \
    .filterDate('1990-09-01', '1990-10-01')

# Map.addLayer(collection, {}, "Collection2019")
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
print(collection.aggregate_array('system:index').getInfo())
geemap.ee_export_image_collection(collection, out_dir=out_dir)
geemap.ee_export_image_collection_to_drive(collection, folder='export', scale=10)

## Change detection

In [ ]:
import os
import ee
import geemap
from geemap.basemaps import basemaps

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
LC_2015 = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2015")\
.select('discrete_classification').clip(bf_boarder)
LC_2019 = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019")\
.select('discrete_classification').clip(bf_boarder)

left_layer = geemap.ee_tile_layer(LC_2015, {}, 'LC_2015')
right_layer = geemap.ee_tile_layer(LC_2019, {}, 'LC_2019')

Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.split_map(left_layer, right_layer)
Map

## Zonal Statistics

### Export zonal statistics to csv, shp, json, kml, kmz

In [ ]:
import ee
import geemap
import os

In [ ]:
# Create the interactive map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
dem_vis = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Engine DEM to map
Map.addLayer(dem, dem_vis, 'SRTM DEM')

# Add Landsat data to map
landsat = ee.Image('LE7_TOA_5YEAR/1999_2003')

landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.4
}
Map.addLayer(landsat, landsat_vis, "LE7_TOA_5YEAR/1999_2003")


Africa = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'Africa'))
Map.addLayer(Africa, {}, 'Africa')

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_dem_africa = os.path.join(out_dir, 'dem_africa.csv')  

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(dem, Africa, out_dem_africa, statistics_type='MEAN', scale=1000)

In [ ]:
out_landsat_africa = os.path.join(out_dir, 'landsat_africa.shp')  
geemap.zonal_statistics(landsat, Africa, out_landsat_africa, statistics_type='SUM', scale=1000)

In [ ]:
geemap.create_download_link(out_dem_africa)

In [ ]:
geemap.create_download_link(out_landsat_africa)

### Zonal Statistics by group

In [ ]:
# Create the interactive map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
landcover = ee.Image('MODIS/051/MCD12Q1/2013_01_01') \
    .select('Land_Cover_Type_1')

Map.setCenter(12.2395,  -1.5584, 8)
Map.addLayer(landcover, {}, 'MODIS Land Cover')

In [ ]:
Map.add_legend(builtin_legend='MODIS/051/MCD12Q1')

In [ ]:
Africa = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'Africa'))
Map.addLayer(Africa, {}, 'Africa')

In [ ]:
# output sum
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
nlcd_africa_sum = os.path.join(out_dir, 'nlcd_africa_sum.csv')  

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
# statistics_type can be either 'SUM' or 'PERCENTAGE'
# denominator can be used to convert square meters to other areal units, such as square kilimeters
geemap.zonal_statistics_by_group(landcover, Africa, nlcd_africa_sum, statistics_type='SUM', denominator=1000000, decimal_places=2)

In [ ]:
# output percentage
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
nlcd_africa_percentage = os.path.join(out_dir, 'nlcd_africa_percentage.csv')  

# statistics_type can be either 'SUM' or 'PERCENTAGE'
# denominator can be used to convert square meters to other areal units, such as square kilimeters
geemap.zonal_statistics_by_group(landcover, Africa, nlcd_africa_percentage, statistics_type='PERCENTAGE', denominator=1000000, decimal_places=2)

In [ ]:
geemap.create_download_link(nlcd_africa_sum)

In [ ]:
geemap.create_download_link(nlcd_africa_sum)

# Remote sensing data

Raster data are represented as Image objects in Earth Engine. Images are composed of one or more bands and each band has its own name, data type, scale, mask and projection. Each image has metadata stored as a set of properties.

In addition to loading images from the archive by an image ID, images can also be created from constants, lists or other suitable Earth Engine objects. The following illustrates methods for creating images, getting band subsets, and manipulating bands.

More information about ee.Image can be found in the Earth Engine Documentation.

https://developers.google.com/earth-engine/guides/image_overview

In [ ]:
# !pip isntall geemap

## Loading a single-band image
Images can be loaded by pasting an Earth Engine asset ID into the ee.Image constructor. The image IDs can be found in data catalog. https://developers.google.com/earth-engine/datasets

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
dem = ee.Image('CGIAR/SRTM90_V4').clip(bf_boarder)
Map.addLayer(dem, {}, "DEM")

In [ ]:
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 600,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
Map.addLayer(dem, vis_params, "DEM Vis")

## Loading a multi-band image

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
          .filterDate('2020-10-01', '2020-12-30') \
          .filterMetadata('CLOUD_COVER', 'less_than', 10) \
          .sort("CLOUD_COVER") \
           .filterBounds(bf_boarder) 

# Center the map and display the image.
Map.centerObject(image, zoom=8)

vis_params = {'bands': ['B5', 'B4', 'B3'],
              'min': 0.0,
              'max': 3000,
              'opacity': 1.0,
              'gamma': 1.2}
Map.addLayer(image, vis_params, 'Landsat')

## Select the best image in each year

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(11.1464, -3.05784), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
Dano = ee.Geometry.Point(-3.05784, 11.1464)

In [ ]:
def best_image(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, 'year')
    
    image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(Dano) \
        .filterDate(start_date, end_date) \
        .sort("CLOUD_COVER") \
        .first()
    
    return image

In [ ]:
start_year = 2013
end_year = 2020
years = ee.List.sequence(start_year, end_year)
year_list = years.getInfo()
print(year_list)

In [ ]:
images = years.map(best_image)

In [ ]:
count = images.size().getInfo()
print(count)

In [ ]:
ee.ImageCollection(images).aggregate_array("CLOUD_COVER").getInfo()

In [ ]:
for index in range(0, count):
    image = ee.Image(images.get(index))
    layer_name = "Image " + str(year_list[index])
    Map.addLayer(image, vis_params, layer_name, False)

## Cloud free composite

In [ ]:
import ee
import geemap
ee.Initialize()

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
years = ee.List.sequence(2013, 2020)
years.getInfo()

In [ ]:
def yearly_image(year):
    
    start_date = ee.Date.fromYMD(year, 1, 1) 
    end_date = start_date.advance(1, "year")
    
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
        .filterDate(start_date, end_date) \
        .filterBounds(bf_boarder) 
    
    image =  ee.Algorithms.Landsat.simpleComposite(collection).clipToCollection(bf_boarder)

    return image

In [ ]:
images = years.map(yearly_image)

In [ ]:
vis_params = {'bands': ['B5',  'B4',  'B3'], 'max': 128}

In [ ]:
for index in range(0, 8):
    image = ee.Image(images.get(index))
    layer_name = "Image " + str(index + 2013)
    Map.addLayer(image, vis_params, layer_name)

## Normalized difference vegetation index (NDVI) and Enhanced vegetation index (EVI)

### Landsat

#### Color palettes
To display a single band of an image in color, set the parameter with a color ramp represented by a list of CSS-style color strings. 

The following example illustrates how to use colors to render a Normalized Difference Vegetation Index (NDVI) image.

In [ ]:
import ee
import geemap

In [ ]:
# Create default map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
# Load landsat image
imageCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
                    .filterDate('2000-10-01', '2020-12-31') \
                    .filterBounds(bf_boarder)

##### Landsat NDVI
1. Use function "image.normalizedDifference"
2. Use add(), subtract(), and divide() operators.

##### Landsat NDVI: average all month
Results in a image collection containing 12 images as long term mean

In [ ]:
startyear = 2000
endyear = 2020

In [ ]:
startDate=ee.Date.fromYMD(startyear,1,1)
endDate=ee.Date.fromYMD(endyear+1,1,1)

In [ ]:
months = ee.List.sequence(1, 12)

In [ ]:
def func_ndvi(m):
  filtered = imageCollection.filter(ee.Filter.calendarRange(**{
    'start': m,
    'field': 'month'
  }))
  composite = ee.Algorithms.Landsat.simpleComposite(filtered)
  return composite.normalizedDifference(['B4', 'B3']).rename('NDVI') \
      .set('month', m)

In [ ]:
composites = ee.ImageCollection.fromImages(months.map(func_ndvi))

In [ ]:
# print(composites.size())

In [ ]:
ndvi_first = ee.Image(composites.first().clip(bf_boarder))
ndvi_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
ndviViz = {'min': 0, 'max': 0.3 ,'palette':ndvi_palette}
Map.addLayer(ndvi_first, ndviViz, 'check ndvi')

##### Landsat NDVI: each year to calculate anomalies

In [ ]:
# Load landsat image
imageCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
                    .filterDate('2000-10-01', '2020-12-31') \
                    .filterBounds(bf_boarder)

In [ ]:
startyear = 2000
endyear = 2020
year = startyear

In [ ]:
startDate=ee.Date.fromYMD(year,1,1)
endDate=ee.Date.fromYMD(year,12,31)

In [ ]:
months = ee.List.sequence(1, 12)

In [ ]:
def func_ndvi(m):
  filtered = imageCollection.filter(ee.Filter.calendarRange(**{
    'start': m,
    'field': 'month'
  }))
  composite = ee.Algorithms.Landsat.simpleComposite(filtered)
  return composite.normalizedDifference(['B4', 'B3']).rename('NDVI') \
      .set('month', m)

In [ ]:
for year in range(startyear, endyear+1, 1):
    startDate=ee.Date.fromYMD(year,1,1)
    endDate=ee.Date.fromYMD(year,12,31)
    imageCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
                    .filterDate(startDate, endDate) \
                    .filterBounds(bf_boarder)
    composites = ee.ImageCollection.fromImages(months.map(func_ndvi))

In [ ]:
ndvi_first = ee.Image(composites.first().clip(bf_boarder))
ndvi_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
ndviViz = {'min': 0, 'max': 0.3 ,'palette':ndvi_palette}
Map.addLayer(ndvi_first, ndviViz, 'ndvi first')

#### Landsat NDVI APP

In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets

In [ ]:
style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:',
    value='Landsat Timelapse',
    width=200,
    style=style
)

In [ ]:
start_year = widgets.IntSlider(description='Start Year:', value=2013, min=2013, max=2021, style=style)

In [ ]:
end_year = widgets.IntSlider(description='End Year:', value=2021, min=2013, max=2021, style=style)

In [ ]:
start_month = widgets.Dropdown(
    description='Start Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='04',
    style=style
)

In [ ]:
end_month = widgets.Dropdown(
    description='End Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='05',
    style=style
)

In [ ]:
start_day = widgets.Dropdown(
    description='Start Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='11',
    style=style
)

In [ ]:
end_day = widgets.Dropdown(
    description='End Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='18',
    style=style
)

In [ ]:
hbox_start = widgets.HBox([start_year, start_month, start_day])
hbox_start

In [ ]:
hbox_end = widgets.HBox([end_year, end_month, end_day])
hbox_end

In [ ]:
start_date = str(start_year.value) + '-' + str(start_month.value) + '-' + str(start_day.value)
end_date = str(end_year.value) + '-' + str(end_month.value) + '-' + str(end_day.value)
print(start_date)
print(end_date)

In [ ]:
ee.Initialize()
# Create default map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
# Load landsat image
imageCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
                    .filterDate(start_date, end_date) \
                    .filterBounds(bf_boarder)

In [ ]:
# Function to calculate and add an NDVI band
def addNDVI(image):
   return image.addBands(image.normalizedDifference(['B5', 'B4']))

In [ ]:
# Add NDVI band to image collection
imageCollection = imageCollection.map(addNDVI)

In [ ]:
NDVI = ee.Image(imageCollection.select('nd').mean().clip(bf_boarder))
NDVI_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
NDVIViz = {'min': 0, 'max': 0.35 ,'palette':NDVI_palette}
Map.addLayer(NDVI, NDVIViz, start_date + 'to' + end_date)

##### Exporting all bands as one single image

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, start_date+'to'+end_date+'.tif')

In [ ]:
geemap.ee_export_image(NDVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=False)

##### Exporting each band as one image

In [ ]:
geemap.ee_export_image(NDVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=True)

##### Exporting an image to Google Drive

In [ ]:
# geemap.ee_export_image_to_drive(NDVI, description=start_date + 'to' + end_date, folder='export', region=bf_boarder, scale=30)

##### Exporting maps as HTML

In [ ]:
# Exporting maps as HTML
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'Myhtml.html')
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [ ]:
Map.to_html(outfile=html_file, title= 'NDVI', width='100%', height='880px')

##### Exporting maps as PNG/JPG

In [ ]:
png_file = os.path.join(download_dir, 'my_map.png')
Map.to_image(outfile=png_file, monitor=1)

In [ ]:
jpg_file = os.path.join(download_dir, 'my_map.jpg')
Map.to_image(outfile=jpg_file, monitor=1)

##### Landsat EVI
##### Landsat EVI: average all month

In [ ]:
months = ee.List.sequence(1, 12)

In [ ]:
def func_evi(m):
  filtered = imageCollection.filter(ee.Filter.calendarRange(**{
    'start': m,
    'field': 'month'
  }))
  composite = ee.Algorithms.Landsat.simpleComposite(filtered)
  evi = composite.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': composite.select('B5'),
      'RED': composite.select('B4'),
      'BLUE': composite.select('B2') 
  }).rename('EVI').set('month', m)
  return evi

In [ ]:
composites = ee.ImageCollection.fromImages(months.map(func_evi))

In [ ]:
# print(composites)

In [ ]:
evi_first = ee.Image(composites.first().clip(bf_boarder))
evi_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
eviViz = {'min': -5, 'max': 2 ,'palette':evi_palette}
Map.addLayer(evi_first, eviViz, 'evi first')

##### Landsat EVI: each year to calculate anomalies

In [ ]:
# Load landsat image
imageCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
                    .filterDate('2000-10-01', '2020-12-31') \
                    .filterBounds(bf_boarder)

In [ ]:
startyear = 2000
endyear = 2020
year = startyear

In [ ]:
startDate=ee.Date.fromYMD(year,1,1)
endDate=ee.Date.fromYMD(year,12,31)

In [ ]:
months = ee.List.sequence(1, 12)

In [ ]:
def func_evi(m):
  filtered = imageCollection.filter(ee.Filter.calendarRange(**{
    'start': m,
    'field': 'month'
  }))
  composite = ee.Algorithms.Landsat.simpleComposite(filtered)
  evi = composite.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': composite.select('B5'),
      'RED': composite.select('B4'),
      'BLUE': composite.select('B2') 
  }).rename('EVI').set('month', m)
  return evi

In [ ]:
for year in range(startyear, endyear+1, 1):
    startDate=ee.Date.fromYMD(year,1,1)
    endDate=ee.Date.fromYMD(year,12,31)
    imageCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
                    .filterDate(startDate, endDate) \
                    .filterBounds(bf_boarder)
    composites = ee.ImageCollection.fromImages(months.map(func_evi))

In [ ]:
evi_first = ee.Image(composites.first().clip(bf_boarder))
evi_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
eviViz = {'min': -5, 'max': 2 ,'palette':evi_palette}
Map.addLayer(evi_first, eviViz, 'evi first')

#### Landsat EVI APP

In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets

In [ ]:
geemap.ee_export_image?

In [ ]:
style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:',
    value='Landsat Timelapse',
    width=200,
    style=style
)

In [ ]:
start_year = widgets.IntSlider(description='Start Year:', value=2013, min=2013, max=2021, style=style)

In [ ]:
end_year = widgets.IntSlider(description='End Year:', value=2021, min=2013, max=2021, style=style)

In [ ]:
start_month = widgets.Dropdown(
    description='Start Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='04',
    style=style
)

In [ ]:
end_month = widgets.Dropdown(
    description='End Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='05',
    style=style
)

In [ ]:
start_day = widgets.Dropdown(
    description='Start Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='11',
    style=style
)

In [ ]:
end_day = widgets.Dropdown(
    description='End Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='18',
    style=style
)

In [ ]:
hbox_start = widgets.HBox([start_year, start_month, start_day])
hbox_start

In [ ]:
hbox_end = widgets.HBox([end_year, end_month, end_day])
hbox_end

In [ ]:
start_date = str(start_year.value) + '-' + str(start_month.value) + '-' + str(start_day.value)
end_date = str(end_year.value) + '-' + str(end_month.value) + '-' + str(end_day.value)
print(start_date)
print(end_date)

In [ ]:
ee.Initialize()
# Create default map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
# Load landsat image
imageCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
                    .filterDate(start_date, end_date) \
                    .filterBounds(bf_boarder)

In [ ]:
# Function to calculate and add an EVI band
def addEVI(image):
    EVI = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR' : image.select('B5'),
      'RED' : image.select('B4'),
      'BLUE': image.select('B2')}).rename('EVI')
    return image.addBands(EVI)

In [ ]:
# Add NDVI band to image collection
imageCollection = imageCollection.map(addEVI)

In [ ]:
# Map.addLayer(imageCollection.first().clip(bf_boarder), {}, 'first')

In [ ]:
EVI = ee.Image(imageCollection.select('EVI').mean().clip(bf_boarder))
EVI_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
EVIViz = {'min': -0.3, 'max': 1 ,'palette':EVI_palette}
Map.addLayer(EVI, EVIViz, start_date + 'to' + end_date)

##### Exporting all bands as one single image

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'EVI' + start_date+'to'+end_date+'.tif')

In [ ]:
geemap.ee_export_image(EVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=False)

##### Exporting each band as one image

In [ ]:
geemap.ee_export_image(NDVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=True)

##### Exporting an image to Google Drive

In [ ]:
# geemap.ee_export_image_to_drive(NDVI, description=start_date + 'to' + end_date, folder='export', region=bf_boarder, scale=30)

##### Exporting maps as HTML

In [ ]:
# Exporting maps as HTML
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'Myhtml.html')
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [ ]:
Map.to_html(outfile=html_file, title= 'NDVI', width='100%', height='880px')

##### Exporting maps as PNG/JPG

In [ ]:
png_file = os.path.join(download_dir, 'my_map.png')
Map.to_image(outfile=png_file, monitor=1)

In [ ]:
jpg_file = os.path.join(download_dir, 'my_map.jpg')
Map.to_image(outfile=jpg_file, monitor=1)

#### Masking
Use image.updateMask() to set the opacity of individual pixels based on where pixels in a mask image are non-zero. Pixels equal to zero in the mask are excluded from computations and the opacity is set to 0 for display.

The following example uses an NDWI threshold to update the mask on the NDWI layer created previously.

In [ ]:
# Mask the non-vegetation parts of the image, where NDVI < 0.4.
ndviMasked = ndvi_first.updateMask(ndvi_first.gte(0.4))
Map.addLayer(ndviMasked, ndviViz, 'NDVI masked')

In [ ]:
# Mask the non-vegetation parts of the image, where EVI < 0.5.
eviMasked = evi_first.updateMask(evi_first.gte(0.5))
Map.addLayer(eviMasked, ndviViz, 'EVI masked')

#### Visualization images
Use the image.visualize() method to convert an image into an 8-bit RGB image for display or export. For example, to convert the false-color composite and NDVI to 3-band display images.

In [ ]:
# Create visualization layers.
imageRGB = image.visualize(**{'bands': ['B5', 'B4', 'B3'],
              'min': 0.0,
              'max': 3000,
              'opacity': 1.0,
              'gamma': 1.2})
ndviRGB = ndviMasked.visualize(**{
  'min': 0.5,
  'max': 1,
  'palette': ['eaff00', 'aeff00', '59ff00', '00ff04', '5ac773', '64e366', '00FF00']})

Map.addLayer(imageRGB, {}, 'imageRGB', False)
Map.addLayer(ndviRGB, {}, 'ndviRGB', False)

In [ ]:
eviRGB = eviMasked.visualize(**{
  'palette': ['FF0000', '00FF00']})
Map.addLayer(eviRGB, {}, 'eviRGB', False)

#### Mosaicking
The mosaic() method renders layers in the output image according to their order in the input collection. The following example uses mosaic() to combine the masked NDVI and the false color composite and obtain a new visualization.

In [ ]:
# Mosaic the visualization layers and display (or export).
ndvi_mosaic = ee.ImageCollection([imageRGB, ndviRGB]).mosaic()
Map.addLayer(ndvi_mosaic, {}, 'ndvi mosaic');

In [ ]:
# Mosaic the visualization layers and display (or export).
evi_mosaic = ee.ImageCollection([imageRGB, eviRGB]).mosaic()
Map.addLayer(evi_mosaic, {}, 'evi mosaic');

#### Clipping
The image.clip() method is useful for achieving cartographic effects.

In [ ]:
# Clip the mosaic by Burkina Faso boundary
ndvi_clipped = ndvi_mosaic.clip(bf_boarder)
evi_clipped = evi_mosaic.clip(bf_boarder)
Map.addLayer(ndvi_clipped, {}, 'ndvi_clipped')
Map.addLayer(evi_clipped, {}, 'evi_clipped')

#### Extract urbanized area

In [ ]:
import ee
import geemap

In [ ]:
# Create default map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
# Load landsat image
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
          .filterDate('2000-01-01', '2020-12-31') \
          .filterBounds(bf_boarder) 

##### Landsat NDWI: average all months

In [ ]:
startyear = 2000
endyear = 2020
year = startyear

In [ ]:
startDate=ee.Date.fromYMD(year,1,1)
endDate=ee.Date.fromYMD(year,12,31)

In [ ]:
months = ee.List.sequence(1, 12)

In [ ]:
def func_ndwi(m):
  filtered = imageCollection.filter(ee.Filter.calendarRange(**{
    'start': m,
    'field': 'month'
  }))
  composite = ee.Algorithms.Landsat.simpleComposite(filtered)
  return composite.normalizedDifference(['B3', 'B5']).rename('NDWI') \
      .set('month', m)

In [ ]:
composites_ndwi = ee.ImageCollection.fromImages(months.map(func_ndwi))

In [ ]:
composites_ndvi = ee.ImageCollection.fromImages(months.map(func_ndvi))

In [ ]:
ndwi = composites_ndwi.first()
ndvi = composites_ndvi.first()

To perform per-pixel comparisons between images, use relational operators. To extract urbanized areas in an image, this example uses relational operators to threshold spectral indices, combining the thresholds with And():

In [ ]:
# Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

In [ ]:
# Mask and display the binary layer.
Map.addLayer(bare.selfMask(), {}, 'bare')

This example creates zones of urbanization in a nighttime lights image using relational operators and image.add():

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
# Load a 2014 nightlights image.
nl2013 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182013') \
           .clip(bf_boarder)
lights = nl2013.select('stable_lights')
Map.addLayer(lights, {}, 'Nighttime lights')

In [ ]:
# Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62)).selfMask()

In [ ]:
# Display the thresholded image as three distinct zones near Paris.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
Map.addLayer(zones, {'min': 0, 'max': 3, 'palette': palette}, 'development zones')

In [ ]:
# Create zones using an expression, display.
zonesExp = nl2013.expression(
    "(b('stable_lights') > 62) ? 3" +
      ": (b('stable_lights') > 55) ? 2" +
        ": (b('stable_lights') > 30) ? 1" +
          ": 0"
)
zonesExp = zonesExp.clip(bf_boarder)
Map.addLayer(zonesExp.selfMask(), {'min': 0, 'max': 3, 'palette': palette}, 'development zones (ternary)')

### Sentinel-2 Data

##### Sentinel-2 NDVI

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=6, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
# information on cloud
def maskcloud1(image):
   QA60 = image.select(['QA60'])
   return image.updateMask(QA60.lt(20))

In [ ]:
S2 = ee.ImageCollection("COPERNICUS/S2") \
       .filterBounds(bf_boarder) \
       .filterDate('2015-06-24', '2020-12-31') \
       .map(maskcloud1)

In [ ]:
# Function to calculate and add an NDVI band
def addNDVI(image):
   return image.addBands(image.normalizedDifference(['B8', 'B4']))

In [ ]:
# Add NDVI band to image collection
S2 = S2.map(addNDVI)

In [ ]:
# Extract NDVI band and choose NDVI median image for a period
NDVI = S2.select(['nd'])
ndvi_check = NDVI.filterDate('2016-10-01', '2016-10-30').mean().clip(bf_boarder)

In [ ]:
ndvi_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
ndviViz = {'min': 0, 'max': 0.5 ,'palette':ndvi_palette}
Map.addLayer(ndvi_check, ndviViz, 'ndvi')

#### Sentinel-2 NDVI APP

In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets

In [ ]:
style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:',
    value='Landsat Timelapse',
    width=200,
    style=style
)

In [ ]:
start_year = widgets.IntSlider(description='Start Year:', value=2015, min=2015, max=2021, style=style)

In [ ]:
end_year = widgets.IntSlider(description='End Year:', value=2021, min=2015, max=2021, style=style)

In [ ]:
start_month = widgets.Dropdown(
    description='Start Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='06',
    style=style
)

In [ ]:
end_month = widgets.Dropdown(
    description='End Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='05',
    style=style
)

In [ ]:
start_day = widgets.Dropdown(
    description='Start Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='23',
    style=style
)

In [ ]:
end_day = widgets.Dropdown(
    description='End Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='23',
    style=style
)

In [ ]:
hbox_start = widgets.HBox([start_year, start_month, start_day])
hbox_start

In [ ]:
hbox_end = widgets.HBox([end_year, end_month, end_day])
hbox_end

In [ ]:
start_date = str(start_year.value) + '-' + str(start_month.value) + '-' + str(start_day.value)
end_date = str(end_year.value) + '-' + str(end_month.value) + '-' + str(end_day.value)
print(start_date)
print(end_date)

In [ ]:
ee.Initialize()
# Create default map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
# information on cloud
def maskcloud1(image):
   QA60 = image.select(['QA60'])
   return image.updateMask(QA60.lt(20))

In [ ]:
S2 = ee.ImageCollection("COPERNICUS/S2") \
       .filterBounds(bf_boarder) \
       .filterDate(start_date, end_date) \
       .map(maskcloud1)

In [ ]:
# Function to calculate and add an NDVI band
def addNDVI(image):
   return image.addBands(image.normalizedDifference(['B8', 'B4']))

In [ ]:
# Add NDVI band to image collection
S2 = S2.map(addNDVI)

In [ ]:
NDVI = ee.Image(S2.select('nd').mean().clip(bf_boarder))
NDVI_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
NDVIViz = {'min': 0, 'max': 0.4 ,'palette':NDVI_palette}
Map.addLayer(NDVI, NDVIViz, start_date + 'to' + end_date)

##### Exporting all bands as one single image

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, start_date+'to'+end_date+'.tif')

In [ ]:
geemap.ee_export_image(NDVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=False)

##### Exporting each band as one image

In [ ]:
geemap.ee_export_image(NDVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=True)

##### Exporting an image to Google Drive

In [ ]:
# geemap.ee_export_image_to_drive(NDVI, description=start_date + 'to' + end_date, folder='export', region=bf_boarder, scale=30)

##### Exporting maps as HTML

In [ ]:
# Exporting maps as HTML
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'Myhtml.html')
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [ ]:
Map.to_html(outfile=html_file, title= 'NDVI', width='100%', height='880px')

##### Exporting maps as PNG/JPG

In [ ]:
png_file = os.path.join(download_dir, 'my_map.png')
Map.to_image(outfile=png_file, monitor=1)

In [ ]:
jpg_file = os.path.join(download_dir, 'my_map.jpg')
Map.to_image(outfile=jpg_file, monitor=1)

##### Sentinel-2 EVI

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=6, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
# Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = ee.Number(2).pow(10).int()
  cirrusBitMask = ee.Number(2).pow(11).int()

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data.
  return image.updateMask(mask).divide(10000)

S2 = ee.ImageCollection("COPERNICUS/S2") \
       .filterBounds(bf_boarder) \
       .filterDate('2015-06-24', '2020-12-31') 

In [ ]:
# Add EVI using an expression.
def addEVI(image):
    EVI = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR' : image.select('B8'),
      'RED' : image.select('B4'),
      'BLUE': image.select('B2')}).rename('EVI')
    return image.addBands(EVI)

In [ ]:
# Add EVI band to image collection
S2 = S2.map(addEVI)

In [ ]:
# Extract EVI band and choose NDVI median image for a period
EVI = S2.select(['EVI'])
evi_check = EVI.filterDate('2016-10-01', '2016-10-30').mean().clip(bf_boarder)

In [ ]:
evi_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
eviViz = {'min': -5, 'max': 1 ,'palette':evi_palette}
Map.addLayer(evi_check, eviViz, 'evi check')

#### Sentinel-2 EVI APP

In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets

In [ ]:
style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:',
    value='Landsat Timelapse',
    width=200,
    style=style
)

In [ ]:
start_year = widgets.IntSlider(description='Start Year:', value=2015, min=2015, max=2021, style=style)

In [ ]:
end_year = widgets.IntSlider(description='End Year:', value=2021, min=2015, max=2021, style=style)

In [ ]:
start_month = widgets.Dropdown(
    description='Start Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='06',
    style=style
)

In [ ]:
end_month = widgets.Dropdown(
    description='End Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='05',
    style=style
)

In [ ]:
start_day = widgets.Dropdown(
    description='Start Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='23',
    style=style
)

In [ ]:
end_day = widgets.Dropdown(
    description='End Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='23',
    style=style
)

In [ ]:
hbox_start = widgets.HBox([start_year, start_month, start_day])
hbox_start

In [ ]:
hbox_end = widgets.HBox([end_year, end_month, end_day])
hbox_end

In [ ]:
start_date = str(start_year.value) + '-' + str(start_month.value) + '-' + str(start_day.value)
end_date = str(end_year.value) + '-' + str(end_month.value) + '-' + str(end_day.value)
print(start_date)
print(end_date)

In [ ]:
ee.Initialize()
# Create default map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
# information on cloud
def maskcloud1(image):
   QA60 = image.select(['QA60'])
   return image.updateMask(QA60.lt(20))

In [ ]:
# Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = ee.Number(2).pow(10).int()
  cirrusBitMask = ee.Number(2).pow(11).int()

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data.
  return image.updateMask(mask).divide(10000)

In [ ]:
S2 = ee.ImageCollection("COPERNICUS/S2") \
       .filterBounds(bf_boarder) \
       .filterDate(start_date, end_date) \
       .map(maskcloud1)

In [ ]:
# Add EVI using an expression.
def addEVI(image):
    EVI = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR' : image.select('B8'),
      'RED' : image.select('B4'),
      'BLUE': image.select('B2')}).rename('EVI')
    return image.addBands(EVI)

In [ ]:
# Add EVI band to image collection
S2 = S2.map(addEVI)

In [ ]:
EVI = ee.Image(S2.select('EVI').mean().clip(bf_boarder))
EVI_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
EVIViz = {'min': 0, 'max': 0.4 ,'palette':EVI_palette}
Map.addLayer(EVI, EVIViz, start_date + 'to' + end_date)

##### Exporting all bands as one single image

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, start_date+'to'+end_date+'.tif')

In [ ]:
geemap.ee_export_image(EVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=False)

##### Exporting each band as one image

In [ ]:
geemap.ee_export_image(EVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=True)

##### Exporting an image to Google Drive

In [ ]:
# geemap.ee_export_image_to_drive(EVI, description=start_date + 'to' + end_date, folder='export', region=bf_boarder, scale=30)

##### Exporting maps as HTML

In [ ]:
# Exporting maps as HTML
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'Myhtml.html')
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [ ]:
Map.to_html(outfile=html_file, title= 'EVI', width='100%', height='880px')

##### Exporting maps as PNG/JPG

In [ ]:
png_file = os.path.join(download_dir, 'my_map.png')
Map.to_image(outfile=png_file, monitor=1)

In [ ]:
jpg_file = os.path.join(download_dir, 'my_map.jpg')
Map.to_image(outfile=jpg_file, monitor=1)

### Proba-V

In [ ]:
import ee
import geemap

In [ ]:
# Create default map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

##### Proba-V NDVI

In [ ]:
image = ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M') \
          .filterDate('2020-10-01', '2020-12-30') \
          .filterBounds(bf_boarder) 
ndvi = image.select('NDVI').median().clip(bf_boarder)
ndvi_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
Map.addLayer(ndvi, {'min': 0, 'max': 170, 'palette': ndvi_palette}, 'Proba-V ndvi 2020');

#### Proba-V NDVI APP

In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets

In [ ]:
style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:',
    value='Landsat Timelapse',
    width=200,
    style=style
)

In [ ]:
start_year = widgets.IntSlider(description='Start Year:', value=2013, min=2013, max=2021, style=style)

In [ ]:
end_year = widgets.IntSlider(description='End Year:', value=2021, min=2013, max=2021, style=style)

In [ ]:
start_month = widgets.Dropdown(
    description='Start Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='04',
    style=style
)

In [ ]:
end_month = widgets.Dropdown(
    description='End Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='05',
    style=style
)

In [ ]:
start_day = widgets.Dropdown(
    description='Start Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='11',
    style=style
)

In [ ]:
end_day = widgets.Dropdown(
    description='End Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='18',
    style=style
)

In [ ]:
hbox_start = widgets.HBox([start_year, start_month, start_day])
hbox_start

In [ ]:
hbox_end = widgets.HBox([end_year, end_month, end_day])
hbox_end

In [ ]:
start_date = str(start_year.value) + '-' + str(start_month.value) + '-' + str(start_day.value)
end_date = str(end_year.value) + '-' + str(end_month.value) + '-' + str(end_day.value)
# print(start_date)
# print(end_date)

In [ ]:
ee.Initialize()
# Create default map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
image = ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M') \
          .filterDate(start_date, end_date) \
          .filterBounds(bf_boarder) \
          .select('NDVI')

In [ ]:
NDVI = image.median().clip(bf_boarder)
NDVI_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
Map.addLayer(NDVI, {'min': 0, 'max': 170, 'palette': NDVI_palette}, 'Proba-V NDVI');

##### Exporting all bands as one single image

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'EVI' + start_date+'to'+end_date+'.tif')

In [ ]:
geemap.ee_export_image(NDVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=False)

##### Exporting each band as one image

In [ ]:
geemap.ee_export_image(NDVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=True)

##### Exporting an image to Google Drive

In [ ]:
# geemap.ee_export_image_to_drive(NDVI, description=start_date + 'to' + end_date, folder='export', region=bf_boarder, scale=30)

##### Exporting maps as HTML

In [ ]:
# Exporting maps as HTML
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'Myhtml.html')
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [ ]:
Map.to_html(outfile=html_file, title= 'NDVI', width='100%', height='880px')

##### Exporting maps as PNG/JPG

In [ ]:
png_file = os.path.join(download_dir, 'my_map.png')
Map.to_image(outfile=png_file, monitor=1)

In [ ]:
jpg_file = os.path.join(download_dir, 'my_map.jpg')
Map.to_image(outfile=jpg_file, monitor=1)

##### Proba-V EVI

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=6, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
image = ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M') \
          .filterDate('2013-10-18', '2020-12-30') \
          .filterBounds(bf_boarder) 

In [ ]:
# Add EVI using an expression.
def addEVI(image):
    EVI = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR' : image.select('NIR'),
      'RED' : image.select('RED'),
      'BLUE': image.select('BLUE')}).rename('EVI')
    return image.addBands(EVI)

In [ ]:
# Add EVI band to image collection
image = image.map(addEVI)

In [ ]:
# Extract EVI band and choose NDVI median image for a period
EVI = image.select(['EVI'])
evi_check = EVI.filterDate('2016-10-01', '2016-10-30').mean().clip(bf_boarder)

In [ ]:
evi_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
Map.addLayer(evi_check, {'min': -5, 'max': 1, 'palette': evi_palette}, 'Proba-V evi 2020');

#### Proba-V EVI APP

In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets

In [ ]:
style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:',
    value='Landsat Timelapse',
    width=200,
    style=style
)

In [ ]:
start_year = widgets.IntSlider(description='Start Year:', value=2013, min=2013, max=2021, style=style)

In [ ]:
end_year = widgets.IntSlider(description='End Year:', value=2021, min=2013, max=2021, style=style)

In [ ]:
start_month = widgets.Dropdown(
    description='Start Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='04',
    style=style
)

In [ ]:
end_month = widgets.Dropdown(
    description='End Month:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    value='05',
    style=style
)

In [ ]:
start_day = widgets.Dropdown(
    description='Start Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='11',
    style=style
)

In [ ]:
end_day = widgets.Dropdown(
    description='End Day:',
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
             '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
             '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    value='18',
    style=style
)

In [ ]:
hbox_start = widgets.HBox([start_year, start_month, start_day])
hbox_start

In [ ]:
hbox_end = widgets.HBox([end_year, end_month, end_day])
hbox_end

In [ ]:
start_date = str(start_year.value) + '-' + str(start_month.value) + '-' + str(start_day.value)
end_date = str(end_year.value) + '-' + str(end_month.value) + '-' + str(end_day.value)
# print(start_date)
# print(end_date)

In [ ]:
ee.Initialize()
# Create default map
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
image = ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M') \
          .filterDate(start_date, end_date) \
          .filterBounds(bf_boarder) 

In [ ]:
# Add EVI using an expression.
def addEVI(image):
    EVI = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR' : image.select('NIR'),
      'RED' : image.select('RED'),
      'BLUE': image.select('BLUE')}).rename('EVI')
    return image.addBands(EVI)

In [ ]:
# Add EVI band to image collection
image = image.map(addEVI)

In [ ]:
EVI = image.select('EVI').mean().clip(bf_boarder)
EVI_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
EVIViz = {'min': -1, 'max': 1.5 ,'palette':EVI_palette}
Map.addLayer(EVI, EVIViz, start_date + 'to' + end_date)

##### Exporting all bands as one single image

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'EVI' + start_date+'to'+end_date+'.tif')

In [ ]:
geemap.ee_export_image(EVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=False)

##### Exporting each band as one image

In [ ]:
geemap.ee_export_image(EVI, filename=filename, scale=900, region=bf_boarder.geometry(), file_per_band=True)

##### Exporting an image to Google Drive

In [ ]:
# geemap.ee_export_image_to_drive(NDVI, description=start_date + 'to' + end_date, folder='export', region=bf_boarder, scale=30)

##### Exporting maps as HTML

In [ ]:
# Exporting maps as HTML
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'Myhtml.html')
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [ ]:
Map.to_html(outfile=html_file, title= 'EVI', width='100%', height='880px')

##### Exporting maps as PNG/JPG

In [ ]:
png_file = os.path.join(download_dir, 'my_map.png')
Map.to_image(outfile=png_file, monitor=1)

In [ ]:
jpg_file = os.path.join(download_dir, 'my_map.jpg')
Map.to_image(outfile=jpg_file, monitor=1)

## Visualizing time series

### Visualizing weather data

In [ ]:
import ee
import geemap
from geemap import *

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=6, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
Africa_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('wld_rgn', 'Africa'))

bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))
mask = bf_boarder.geometry()

In [ ]:
collection = ee.ImageCollection('NOAA/GFS0P25') \
  .filterDate('2020-11-22', '2020-11-23') \
  .limit(24) \
  .select('temperature_2m_above_ground') \
  .filterBounds(bf_boarder) 


vis_params = {
  'min': -40.0,
  'max': 35.0,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

first_image = collection.first().clip(bf_boarder)

Map.addLayer(first_image, vis_params, "First image")

In [ ]:
image = collection.toBands()
image = image.clipToCollection(bf_boarder)

Map.addLayer(image, {}, "Time series", False)

In [ ]:
labels = [str(n).zfill(2) + ":00" for n in range(0, 24)]

In [ ]:
Map.add_time_slider(collection, vis_params, region=bf_boarder, labels=labels, time_interval=1)

In [ ]:
# Add colorbar
width = 250
height = 30
palette = ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
labels = [-40, 35]
colorbar = create_colorbar(width=width, height=height, palette=palette, vertical=False,
                    add_labels=True, font_size=14, labels=labels)
show_image(colorbar)

### Visualizing vegetation data

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=6, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
collection = ee.ImageCollection('MODIS/MCD43A4_006_NDVI') \
                  .filter(ee.Filter.date('2019-10-01', '2020-11-01')) \
                  .select("NDVI")\
                  .filterBounds(bf_boarder)

vis_params = {
  'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

first_image = collection.first().clip(bf_boarder)

Map.addLayer(first_image, vis_params, "First image")

In [ ]:
image = collection.toBands()
image = image.clipToCollection(bf_boarder)
Map.addLayer(image, {}, "Time series", False)

In [ ]:
labels = collection.aggregate_array("system:index").getInfo()
Map.add_time_slider(collection, vis_params, region=bf_boarder, labels=labels, time_interval=5)

### Optical data timeseries

Source code for landsat and sentinel-2 timeseries:
https://github.com/giswqs/geemap/blob/master/geemap/common.py

#### Landsat timeseries

In [ ]:
import ee
import geemap

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
landsat_ts = geemap.landsat_timeseries(roi=bf_boarder, start_year=1984, end_year=2019, start_date='10-01', end_date='12-31')

In [ ]:
layer_names = ['Landsat ' + str(year) for year in range(1984, 2020)]
print(layer_names)

In [ ]:
landsat_vis = {
    'min': 0,
    'max': 4000,
    'gamma': [1, 1, 1],
    'bands': ['NIR', 'Red', 'Green']}

In [ ]:
Map = geemap.Map()
Map.ts_inspector(left_ts=landsat_ts, right_ts=landsat_ts, left_names=layer_names, right_names=layer_names, left_vis=landsat_vis, right_vis=landsat_vis)
Map.centerObject(bf_boarder, zoom=8)
Map

#### Creating Landsat timelapse animation

In [ ]:
import os
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
collection = geemap.landsat_timeseries(roi=bf_boarder, start_year=1985, end_year=2020, start_date='06-10', end_date='09-20')

##### Save as GIF

In [ ]:
# Define arguments for animation function parameters.
video_args = {
  'dimensions': 768,
  'region': bf_boarder,
  'framesPerSecond': 2,
  'bands': ['NIR', 'Red', 'Green'],
  'min': 0,
  'max': 4000,
  'gamma': [1, 1, 1]
}

In [ ]:
work_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
out_gif = os.path.join(work_dir, "landsat_ts.gif")

In [ ]:
geemap.download_ee_video(collection, video_args, out_gif)

##### Add animated text to GIF

In [ ]:
geemap.show_image(out_gif)

In [ ]:
texted_gif = os.path.join(work_dir, "landsat_ts_text.gif")
geemap.add_text_to_gif(out_gif, texted_gif, xy=('3%', '5%'), text_sequence=1985, font_size=30, font_color='#ffffff', add_progress_bar=False, duration=1000)

In [ ]:
label = 'Burkina Faso landsat'
geemap.add_text_to_gif(texted_gif, texted_gif, xy=('2%', '88%'), text_sequence=label, font_size=30, font_color='#ffffff', progress_bar_color='cyan', duration=1000)

In [ ]:
geemap.show_image(texted_gif)

In [ ]:
geemap.download_ee_video(collection, video_args, texted_gif)

#### Sentinel-2 timeseries

In [ ]:
import os
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
roi = ee.Geometry.Polygon(
        [[[-3.090598913590661, 11.131225424381887],
        [-3.0308607544109734,11.131225424381887],
        [-3.0308607544109734,11.170804254529743],
        [-3.090598913590661,11.170804254529743],
        [-3.090598913590661,11.131225424381887]]], None, False)

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
# information on cloud
def maskcloud1(image):
   QA60 = image.select(['QA60'])
   return image.updateMask(QA60.lt(20))

In [ ]:
#*
 # Function to mask clouds using the Sentinel-2 QA band
 # @param {ee.Image} image Sentinel-2 image
 # @return {ee.Image} cloud masked Sentinel-2 image
 #
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Map the function over one year of data and take the median.
# Load Sentinel-2 TOA reflectance data.
# dataset = ee.ImageCollection('COPERNICUS/S2') \
#                   .filterDate('2015-06-23', '2021-05-23') \
#                   .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
#                   .filterBounds(bf_boarder) \
#                   .map(maskS2clouds)


In [ ]:
# dataset = ee.ImageCollection('COPERNICUS/S2') \
#                   .filterDate('2015-06-23', '2021-05-23') \
#                   .filterBounds(bf_boarder) \
#                   .map(maskcloud1)

# rgbVis = {
#   'min': 0,
#   'max': 3000,
#   'bands': ['B8', 'B4', 'B3'],
# }

# Map.addLayer(dataset.median().clip(bf_boarder), rgbVis, 'RGB')

In [ ]:
dataset = ee.ImageCollection('COPERNICUS/S2') \
                  .filterDate('2015-06-23', '2021-05-23') \
                  .filterBounds(roi) \
                  .map(maskcloud1)

In [ ]:
# dataset.getInfo()

In [ ]:
# Add observation year as a preperty to each image
def addYear(img):
    return img.set('year', ee.Image(img).date().get('year'))

In [ ]:
dataset = dataset.map(addYear)

In [ ]:
# Make a distinct year collection; one image representative per year.
years = ee.List(dataset.aggregate_array('year')).distinct().sort()
years.getInfo()

In [ ]:
# Map over the list of years to build a list of annual image composites.

def func_bkr(year):
  return dataset \
    .filterMetadata('year', 'equals', year) \
    .reduce(ee.Reducer.median()) \
    .set('year', year)

s2CompList = years.map(func_bkr)

# Convert the image List to an ImageCollection.
s2CompCol = ee.ImageCollection.fromImages(s2CompList)

In [ ]:
# s2CompCol.getInfo()

In [ ]:
# gifParams = {
#   'bands': ['B8_median', 'B4_median', 'B3_median'],
#   'min': 0,
#   'max': 3000,
# }
# Map.addLayer(s2CompCol.clip(bf_boarder),gifParams,'2015')

In [ ]:
# # Make a day-of-year sequence from 1 to 365 with a 30-day step.
# doyList = ee.List.sequence(1, 365, 30)

In [ ]:
# # Map over the list of days to build a list of image composites.

# def func_yll(startDoy):
#   # Ensure that startDoy is a number.
#   startDoy = ee.Number(startDoy)

#   # Filter images by date range; starting with the current startDate and
#   # ending 15 days later. Reduce the resulting image collection by median.
#   return s2Col \
#     .filter(ee.Filter.calendarRange(startDoy, startDoy.add(29), 'day_of_year')) \
#     .reduce(ee.Reducer.median())

# s2CompList = doyList.map(func_yll)

# # Convert the image List to an ImageCollection.
# s2CompCol = ee.ImageCollection.fromImages(s2CompList)

In [ ]:
# # Define GIF visualization parameters.
# gifParams = {
#   'region': bf_boarder.geometry(),
#   'dimensions': 600,
#   'framesPerSecond': 2,
#   'bands': ['B8_median', 'B4_median', 'B3_median'],
#   'min': 0,
#   'max': 3000
# }

# # Print the GIF URL to the console.
# print(s2CompCol.getVideoThumbURL(gifParams))

In [ ]:
# Define GIF visualization parameters.
gifParams = {
  'region': roi,
  'dimensions': 600,
  'framesPerSecond': 2,
  'bands': ['B8_median', 'B4_median', 'B3_median'],
  'min': 0,
  'max': 3000
}

# Print the GIF URL to the console.
print(s2CompCol.getVideoThumbURL(gifParams))

##### Save as GIF

In [ ]:
work_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
out_gif = os.path.join(work_dir, "s2_ts.gif")

In [ ]:
geemap.download_ee_video(s2CompCol, gifParams, out_gif)

##### Add animated text to GIF

In [ ]:
geemap.show_image(out_gif)

In [ ]:
texted_gif = os.path.join(work_dir, "s2_ts_text.gif")
geemap.add_text_to_gif(out_gif, texted_gif, xy=('3%', '5%'), text_sequence=2015, font_size=30, font_color='#ffffff', add_progress_bar=False, duration=1000)

In [ ]:
label = 'Burkina Faso Sentinel-2'
geemap.add_text_to_gif(texted_gif, texted_gif, xy=('2%', '88%'), text_sequence=label, font_size=30, font_color='#ffffff', progress_bar_color='cyan', duration=1000)

In [ ]:
geemap.show_image(texted_gif)

In [ ]:
geemap.download_ee_video(s2CompCol, gifParams, texted_gif)

#### Creating Sentinel-2 timelapse animation

##### Save as GIF

In [ ]:
import os
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

aoi = ee.Geometry.Polygon(
    [[[-3.090598913590661, 11.131225424381887],
      [-3.0308607544109734,11.131225424381887],
        [-3.0308607544109734,11.170804254529743],
        [-3.090598913590661,11.170804254529743],
        [-3.090598913590661,11.131225424381887]]], None, False)

In [ ]:
sentinel_2_ts = geemap.sentinel2_timeseries(roi=aoi, start_year=2015, end_year=2020, start_date='06-24', end_date='12-31')

In [ ]:
# Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = ee.Number(2).pow(10).int()
  cirrusBitMask = ee.Number(2).pow(11).int()

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data.
  return image.updateMask(mask).divide(10000)

S2 = ee.ImageCollection("COPERNICUS/S2") \
       .filterBounds(aoi) \
       .filterDate('2020-06-24', '2020-12-31') 

In [ ]:
# Define arguments for animation function parameters.
video_args = {
  'region': aoi,
  'framesPerSecond': 2,
  'bands': ['B8', 'B4', 'B3'],
}

In [ ]:
work_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
out_gif = os.path.join(work_dir, "s2_ts.gif")

In [ ]:
geemap.download_ee_video(S2, video_args, out_gif)

In [ ]:
geemap.download_ee_video(sentinel_2_ts, video_args, out_gif)

In [ ]:
geemap.show_image(out_gif)

##### Add animated text to GIF

In [ ]:
texted_gif = os.path.join(work_dir, "s2_ts_text.gif")
geemap.add_text_to_gif(out_gif, texted_gif, xy=('3%', '5%'), text_sequence=1985, font_size=30, font_color='#ffffff', add_progress_bar=False, duration=1000)

In [ ]:
label = 'Burkina Faso landsat'
geemap.add_text_to_gif(texted_gif, texted_gif, xy=('2%', '88%'), text_sequence=label, font_size=30, font_color='#ffffff', progress_bar_color='cyan', duration=1000)

In [ ]:
geemap.show_image(texted_gif)

#### Proba-V timeseries

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
def year_image(year):
    start_date = ee.Date.fromYMD(year, 10, 1)
    end_date = start_date.advance(1, 'year')
    
    image = ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M') \
        .filterBounds(bf_boarder) \
        .filterDate(start_date, end_date) \
        .sort("CLOUD_COVER_PERCENTAGE") \
        .first()
    
    return image

In [ ]:
start_year = 2013
end_year = 2020
years = ee.List.sequence(start_year, end_year)
year_list = years.getInfo()
print(year_list)

In [ ]:
images = years.map(year_image)
count = images.size().getInfo()

In [ ]:
ee.ImageCollection(images).aggregate_array("CLOUD_COVER_PERCENTAGE").getInfo()

In [ ]:
vis_params = {
    'min': 20.0,
    'max': 2000.0,
    'bands' :['RED', 'NIR', 'BLUE']
}

In [ ]:
for index in range(0, count):
    image = ee.Image(images.get(index))
    layer_name = "Image " + str(year_list[index])
    Map.addLayer(image, vis_params, layer_name, False)

# Classification

##### Import libraries and create interactive map

In [ ]:
# Import libraries
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

##### Research area

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

## Using S2 

In [ ]:
# Import libraries
import ee
import geemap


In [ ]:
# ee.Initialize()

In [ ]:
Map = geemap.Map(center=(12.2395,  -1.5584), zoom=10, lite_mode=False) 
Map.add_basemap('HYBRID')
Map

In [ ]:
bf_boarder = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
               .filter(ee.Filter.eq('country_na', 'Burkina Faso'))

In [ ]:
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Map the function over one year of data and take the median.
# Load Sentinel-2 TOA reflectance data.
dataset = ee.ImageCollection('COPERNICUS/S2') \
                  .filterDate('2019-01-01', '2019-12-31') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                  .map(maskS2clouds) \
                  .filterBounds(bf_boarder)

BF_data = dataset.median()

##### Load classification map

In [ ]:
# Convert the google earth engine class table to legend
Map = geemap.Map()

ee_class_table = """

Value	Color	Description
0	282828	Unknown. No or not enough satellite data available.
20	FFBB22	Shrubs. Woody perennial plants with persistent and woody stems and without any defined main stem being less than 5 m tall. The shrub foliage can be either evergreen or deciduous.
30	FFFF4C	Herbaceous vegetation. Plants without persistent stem or shoots above ground and lacking definite firm structure. Tree and shrub cover is less than 10 %.
40	F096FF	Cultivated and managed vegetation / agriculture. Lands covered with temporary crops followed by harvest and a bare soil period (e.g., single and multiple cropping systems). Note that perennial woody crops will be classified as the appropriate forest or shrub land cover type.
50	FA0000	Urban / built up. Land covered by buildings and other man-made structures.
60	B4B4B4	Bare / sparse vegetation. Lands with exposed soil, sand, or rocks and never has more than 10 % vegetated cover during any time of the year.
70	F0F0F0	Snow and ice. Lands under snow or ice cover throughout the year.
80	0032C8	Permanent water bodies. Lakes, reservoirs, and rivers. Can be either fresh or salt-water bodies.
90	0096A0	Herbaceous wetland. Lands with a permanent mixture of water and herbaceous or woody vegetation. The vegetation can be present in either salt, brackish, or fresh water.
100	FAE6A0	Moss and lichen.
111	58481F	Closed forest, evergreen needle leaf. Tree canopy >70 %, almost all needle leaf trees remain green all year. Canopy is never without green foliage.
112	009900	Closed forest, evergreen broad leaf. Tree canopy >70 %, almost all broadleaf trees remain green year round. Canopy is never without green foliage.
113	70663E	Closed forest, deciduous needle leaf. Tree canopy >70 %, consists of seasonal needle leaf tree communities with an annual cycle of leaf-on and leaf-off periods.
114	00CC00	Closed forest, deciduous broad leaf. Tree canopy >70 %, consists of seasonal broadleaf tree communities with an annual cycle of leaf-on and leaf-off periods.
115	4E751F	Closed forest, mixed.
116	007800	Closed forest, not matching any of the other definitions.
121	666000	Open forest, evergreen needle leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs and grassland, almost all needle leaf trees remain green all year. Canopy is never without green foliage.
122	8DB400	Open forest, evergreen broad leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs and grassland, almost all broadleaf trees remain green year round. Canopy is never without green foliage.
123	8D7400	Open forest, deciduous needle leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs and grassland, consists of seasonal needle leaf tree communities with an annual cycle of leaf-on and leaf-off periods.
124	A0DC00	Open forest, deciduous broad leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs and grassland, consists of seasonal broadleaf tree communities with an annual cycle of leaf-on and leaf-off periods.
125	929900	Open forest, mixed.
126	648C00	Open forest, not matching any of the other definitions.
200	000080	Oceans, seas. Can be either fresh or salt-water bodies.

"""

In [ ]:
lc = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification')

# dataset = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global").select('discrete_classification')

vis_params = {
  'palette': ['282828', 'FFBB22', 'FFFF4C', 'F096FF', 'FA0000', 'B4B4B4', 'F0F0F0', '0032C8', '0096A0', 'FAE6A0', '58481F', '009900', '70663E', '00CC00', '4E751F', '007800', '666000', '8DB400', '8D7400', 'A0DC00', '929900', '648C00', '000080']}

lc_raw = lc.clip(bf_boarder)

Map.setCenter(-3.06244, 11.146104)
Map.addLayer(lc_raw, vis_params, 'LC_BF')
Map

##### Prepare for consecutive class labels

In [ ]:
raw_class_values = lc_raw.get('discrete_classification_class_values').getInfo()
class_palette = lc_raw.get('discrete_classification_class_palette').getInfo()


In [ ]:
n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

In [ ]:
lc_new = lc_raw.remap(raw_class_values, new_class_values).select(['remapped'], ['discrete_classification'])
Map.setCenter(-3.06244, 11.146104)
Map.addLayer(lc_new, vis_params, 'LC_BF_new')
Map

In [ ]:
# Set classification values and palette
lc_new = lc_new.set('discrete_classification_class_values', new_class_values)
lc_new = lc_new.set('discrete_classification_class_palette', class_palette)

# Check classification values and palette
lc_class_values = lc_new.get('discrete_classification_class_values').getInfo()
lc_class_palette = lc_new.get('discrete_classification_class_palette').getInfo()


In [ ]:
Map.addLayer(lc_new, {}, 'lc_new_bf')
Map

##### Make training data

In [ ]:
# Making the training dataset
points = lc_new.sample(**{
    'region': bf_boarder,
    'scale': 30,
    'numPixels': 10000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)
Map

##### Split training and testing

In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12']

# This property of the table stores the land cover labels.
label = 'discrete_classification'

# Overlay the points on the imagery to get training.
sample_BF = BF_data.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Adds a column of deterministic pseudorandom numbers. 
sample_BF = sample_BF.randomColumn()

split = 0.7

training_BF = sample_BF.filter(ee.Filter.lt('random', split))
validation_BF = sample_BF.filter(ee.Filter.gte('random', split))

In [ ]:
#print(validation_BF.first().getInfo())

In [ ]:
training_BF.first().getInfo()

In [ ]:
validation_BF.first().getInfo()

In [ ]:
classifier_BF = ee.Classifier.smileRandomForest(10).train(training_BF, label, bands)


In [ ]:
print(classifier_BF)

##### Classify the image

In [ ]:
# Classify the image with the same bands used for training.
# result_Dano = Dano_data.select(bands).classify(classifier)
result_BF = BF_data.select(bands).classify(classifier_BF)

result_BF = result_BF.clip(bf_boarder)

In [ ]:
# # Display the clusters with random colors.
# Map.addLayer(result_Dano.randomVisualizer(), vis_params, 'Dano_classfied')
Map.addLayer(result_BF.randomVisualizer(), {}, 'BF_classfied')
Map

##### Render categorical map

In [ ]:
class_values = lc_new.get('discrete_classification_class_values').getInfo()
print(class_values)

In [ ]:
class_palette = lc_new.get('discrete_classification_class_palette').getInfo()
print(class_palette)

In [ ]:
landcover_BF = result_BF.set('classification_class_values', class_values)
landcover_BF = landcover_BF.set('classification_class_palette', class_palette)

In [ ]:
# Map.addLayer(landcover_Dano, {}, 'Classification_Dano')
Map.addLayer(landcover_BF, {}, 'Classification_BF')
Map

##### Visualize the reults

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

##### Add legned to the map

In [ ]:
Map.add_legend(builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global')
Map

## Accuracy assessment

### Training dataset

In [ ]:
# train_accuracy = classifier.confusionMatrix()
train_accuracy_BF = classifier_BF.confusionMatrix()
#print(train_accuracy_BF)
train_accuracy_BF.accuracy().getInfo()

In [ ]:
train_accuracy_BF.getInfo()

In [ ]:
#print(train_accuracy_BF.accuracy().getInfo()
print(train_accuracy_BF.accuracy())

In [ ]:
train_accuracy_BF.kappa().getInfo()

In [ ]:
train_accuracy_BF.producersAccuracy().getInfo()

In [ ]:
train_accuracy_BF.consumersAccuracy().getInfo()

### Validation dataset

In [ ]:
# validated = validation.classify(classifier)
validated_BF = validation_BF.classify(classifier_BF)

In [ ]:
validated_BF.first().getInfo()

In [ ]:
test_accuracy_BF = validated_BF.errorMatrix('discrete_classification', 'classification')

In [ ]:
test_accuracy_BF.getInfo()

In [ ]:
test_accuracy_BF.accuracy().getInfo()

In [ ]:
test_accuracy_BF.kappa().getInfo()

In [ ]:
test_accuracy_BF.producersAccuracy().getInfo()

### Download the confusion matrix

In [ ]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy_BF.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy_BF.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy_BF.getInfo())
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy_BF.getInfo())

### Reclassify land cover map

In [ ]:
landcover_BF_2 = landcover_BF.remap(new_class_values, class_values).select(['remapped'], ['classification'])

In [ ]:
landcover_BF_2 = landcover_BF_2.set('classification_class_values', class_values)
landcover_BF_2 = landcover_BF_2.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover_BF_2, {}, 'Final classification result')
Map

### Export the results

In [ ]:
mask = bf_boarder.geometry()
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'final classification.tif')
geemap.ee_export_image(landcover_BF_2, filename=out_file, region=mask, scale=900)

Export the result to Google Drive

In [ ]:
# geemap.ee_export_image_to_drive(landcover_BF_2, description='landcover', folder='export', scale=900